In [17]:
import gensim
import spacy
import numpy as np
from scipy.stats import norm
from wefe.word_embedding_model import WordEmbeddingModel
import matplotlib.pyplot as plt

In [18]:
def cosine_similarity(v1: np.ndarray, v2: np.ndarray) -> float:
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

def compute_association(w: np.ndarray, A: np.ndarray) -> float:
    return np.mean([cosine_similarity(w, a) for a in A])

def compute_attribute_association(X: np.ndarray, A: np.ndarray) -> np.ndarray:
    return np.array([compute_association(x, A) for x in X])

def compute_attribute_association_L2(A: np.ndarray, T: np.ndarray) -> np.ndarray:
    return np.array([compute_association(a, T) for a in A])

def compute_joint_std(X_Associations: np.ndarray, Y_Associations: np.ndarray) -> float:
    return np.std(np.concatenate([X_Associations, Y_Associations]), ddof=1)

def compute_p_value(X_Diff: np.ndarray, Y_Diff: np.ndarray, permutations: int=1000) -> float:
    test_statistic = np.sum(X_Diff) - np.sum(Y_Diff)
    empirical_distribution = np.array([np.random.choice(np.concatenate([X_Diff, Y_Diff]), size=len(X_Diff) + len(Y_Diff), replace=False) for _ in range(permutations)])
    empirical_differences = np.sum(empirical_distribution[:, :len(X_Diff)], axis=1) - np.sum(empirical_distribution[:, len(X_Diff):], axis=1)
    
    return 1-norm.cdf(test_statistic, loc=np.mean(empirical_differences), scale=np.std(empirical_differences, ddof=1))

def level_1(X: np.ndarray, Y: np.ndarray, A: np.ndarray, B: np.ndarray, permutations: int=1000) -> float:
    X_Associations_A = compute_attribute_association(X, A)
    X_Associations_B = compute_attribute_association(X, B)
    X_Differential_Associations = X_Associations_A - X_Associations_B

    Y_Associations_A = compute_attribute_association(Y, A)
    Y_Associations_B = compute_attribute_association(Y, B)
    Y_Differential_Associations = Y_Associations_A - Y_Associations_B

    X_Mean = np.mean(X_Differential_Associations)
    Y_Mean = np.mean(Y_Differential_Associations)

    p_value = compute_p_value(X_Differential_Associations, Y_Differential_Associations, permutations=permutations)

    return (X_Mean - Y_Mean) / compute_joint_std(X_Differential_Associations, Y_Differential_Associations), p_value

def level_2(T: np.ndarray, A: np.ndarray, B: np.ndarray, permutations: int=1000) -> float:
    A_Associations_T = compute_attribute_association_L2(A, T)
    B_Associations_T = compute_attribute_association_L2(B, T)

    p_value = compute_p_value(A_Associations_T, B_Associations_T, permutations=permutations)

    return (np.mean(A_Associations_T) - np.mean(B_Associations_T)) / compute_joint_std(A_Associations_T, B_Associations_T), p_value

def level_3(T: np.ndarray, A: np.ndarray) -> float:
    T_Associations_A = [cosine_similarity(t, a) for t in T for a in A]
    return np.mean(T_Associations_A), np.std(T_Associations_A, ddof=1)

def ML_EAT(A: np.ndarray, B: np.ndarray, X: np.ndarray, Y: np.ndarray, permutations: int=1000) -> dict:
    L1_effect_size, L1_p_value = level_1(X, Y, A, B, permutations=permutations)
    L2_effect_size_X, L2_p_value_X = level_2(X, A, B, permutations=permutations)
    L2_effect_size_Y, L2_p_value_Y = level_2(Y, A, B, permutations=permutations)
    L3_mean_AX, L3_std_AX = level_3(X, A)
    L3_mean_BX, L3_std_BX = level_3(X, B)
    L3_mean_AY, L3_std_AY = level_3(Y, A)
    L3_mean_BY, L3_std_BY = level_3(Y, B)

    return {
        'L1_effect_size': L1_effect_size,
        'L1_p_value': L1_p_value,
        'L2_effect_size_X': L2_effect_size_X,
        'L2_p_value_X': L2_p_value_X,
        'L2_effect_size_Y': L2_effect_size_Y,
        'L2_p_value_Y': L2_p_value_Y,
        'L3_mean_AX': L3_mean_AX,
        'L3_std_AX': L3_std_AX,
        'L3_mean_BX': L3_mean_BX,
        'L3_std_BX': L3_std_BX,
        'L3_mean_AY': L3_mean_AY,
        'L3_std_AY': L3_std_AY,
        'L3_mean_BY': L3_mean_BY,
        'L3_std_BY': L3_std_BY,
    }
def get_np_embeddings(target_words: list,
                      vocab_dict: dict,
                      embeddings: np.ndarray) -> np.ndarray:
    """
    Get the embeddings for the target words.
    """

    return np.array([embeddings[vocab_dict[word]] for word in target_words])

In [19]:
# Load pre-trained Dutch FastText embeddings with Gensim
print("Loading word embeddings...")
nl_embeddings = gensim.models.KeyedVectors.load_word2vec_format("/Users/matthijstentije/University/MSc_Data-Science/Thesis/MSc_Data_Science_Thesis/data/cc.nl.300.vec.gz", binary=False)
print("Embeddings loaded.")


Loading word embeddings...
Embeddings loaded.


In [20]:
# Convert to WEFE-compatible format
model = WordEmbeddingModel(nl_embeddings, "Dutch FastText")

In [21]:
# Load spaCy Dutch NLP model
nlp = spacy.load('nl_core_news_lg')


In [22]:
def get_unique_target_words(word, exclude_word, topn=10, unique_n=10):
    """
    Retrieve the closest words (on a embedding level) to a given word, lemmatize them, 
    keep only nouns, remove duplicates, and return the top 'unique_n' words.

    :param word: The target word.
    :param exclude_word: The word to exclude from the results (e.g., 'vrouw' when processing 'man').
    :param topn: Number of similar words to retrieve before deduplication.
    :param unique_n: Final number of unique words to return.
    :return: List of unique lemmatized noun words with similarity scores.
    """
    # Retrieve the most similar words
    similar_words = nl_embeddings.most_similar(word, topn=topn)

    # Dictionary to store unique lemmatized noun words
    lemmatized_nouns = {}
    
    for similar_word, similarity_score in similar_words:
        doc = nlp(similar_word)  # Process with spaCy
        lemma = doc[0].lemma_  # Get the lemma
        pos = doc[0].pos_  # Get the part-of-speech tag
        
        # Exclude words that contain the target word or the opposite gender word
        if word in lemma or exclude_word in lemma:
            continue

        # Only add unique words that are nouns (NOUN or PROPN)
        if lemma not in lemmatized_nouns and pos in ["NOUN", "PROPN"]:
            lemmatized_nouns[lemma] = similarity_score

        # Stop when we have the required number of unique words
        if len(lemmatized_nouns) == unique_n:
            break

    return list(lemmatized_nouns.items())

# Define number of unique words to return
unique_n = 30  

# Get unique lemmatized noun words for "man" (excluding "vrouw") and "vrouw" (excluding "man")
unique_lemmatized_noun_man = get_unique_target_words("man", "vrouw", topn=100, unique_n=unique_n)
unique_lemmatized_noun_vrouw = get_unique_target_words("vrouw", "man", topn=100, unique_n=unique_n)

# Print results
print("Closest unique lemmatized noun words to 'man':")
for word, score in unique_lemmatized_noun_man:
    print(f"{word} (similarity: {score:.4f})")

print("\nClosest unique lemmatized noun words to 'vrouw':")
for word, score in unique_lemmatized_noun_vrouw:
    print(f"{word} (similarity: {score:.4f})")

Closest unique lemmatized noun words to 'man':
kerel (similarity: 0.6592)
dame (similarity: 0.6486)
echtgenoot (similarity: 0.6290)
jongen (similarity: 0.6248)
moeder (similarity: 0.6097)
vriendin (similarity: 0.6005)
echtgenote (similarity: 0.6004)
vader (similarity: 0.5930)
Mannnen (similarity: 0.5897)
dochter (similarity: 0.5763)
schoonvader (similarity: 0.5702)
jongedame (similarity: 0.5590)
zoon (similarity: 0.5587)
persoon (similarity: 0.5587)
schoonmoeder (similarity: 0.5585)
vent (similarity: 0.5572)
knul (similarity: 0.5470)
meisje (similarity: 0.5469)
vriend (similarity: 0.5468)
toiletdaam (similarity: 0.5456)
broer (similarity: 0.5454)
koksjong (similarity: 0.5422)
politieagent (similarity: 0.5403)
mens (similarity: 0.5317)
echtpaar (similarity: 0.5315)
zus (similarity: 0.5289)
rauwdouwer (similarity: 0.5286)
moedert (similarity: 0.5274)
26jaar (similarity: 0.5250)
jongedochter (similarity: 0.5245)

Closest unique lemmatized noun words to 'vrouw':
echtgenoot (similarity: 0.7

## Extract Dutch Adjectives
Now we extract all the lemmas of adjectives in the Dutch language. Therefore, we load in the data downloaded from the Algemeen Nederlands Woordenboek (ANW), a reliable source for Dutch linguistic information.

In [23]:
import pandas as pd
import re

def get_clean_adjectives(file_path_a_o, file_path_o_z, embedding_model, nlp_model):
    """
    Leest twee CSV-bestanden met ANW-resultaten in, maakt de lemmas schoon,
    filtert op woorden met meer dan 1 karakter, verwijdert duplicaten, 
    verwijdert woorden die niet in het embeddingmodel zitten en filtert vervolgens
    op Nederlandse adjectieven via spaCy.

    Parameters:
    - file_path_a_o: pad naar het eerste CSV-bestand (a-o)
    - file_path_o_z: pad naar het tweede CSV-bestand (o-z)
    - embedding_model: een embeddingmodel waarin gecontroleerd wordt of een woord aanwezig is
    - nlp_model: een spaCy-model voor POS-tagging (bijv. "nl_core_news_sm")

    Returns:
    - Een lijst met gefilterde adjectieven.
    """

    def clean_lemma(lemma):
        """
        Zet het lemma om naar lowercase en behoudt alleen de opeenvolgende letters
        aan het begin van de string. Als het lemma niet met een letter begint, retourneert
        de functie een lege string.
        """
        lemma = lemma.lower()
        match = re.match(r"[a-z]+", lemma)
        return match.group(0) if match else ''

    # Lees de twee CSV-bestanden in met de juiste delimiter (puntkomma)
    df_a_o = pd.read_csv(file_path_a_o, delimiter=';')
    df_o_z = pd.read_csv(file_path_o_z, delimiter=';')
    
    # Concateneer de dataframes
    df = pd.concat([df_a_o, df_o_z], ignore_index=True)
    
    # Pas de cleaning-functie toe op de kolom 'lemma'
    df['cleaned_lemma'] = df['lemma'].apply(clean_lemma)
    
    # Zet de verwerkte kolom om in een lijst en verwijder lege strings
    lemma_list = df['cleaned_lemma'].tolist()
    lemma_list = [lemma for lemma in lemma_list if lemma != '']
    
    # Filter op lemma's met meer dan 1 karakter
    filtered_lemmas = [lemma for lemma in lemma_list if len(lemma) > 1]
    
    # Verwijder duplicaten en behoud de volgorde
    unique_lemmas = list(dict.fromkeys(filtered_lemmas))
    print(f"Total unique lemmas: {len(unique_lemmas)}")
    
    # Identificeer woorden die niet in het embeddingmodel zitten
    missing_words = [word for word in unique_lemmas if word not in embedding_model]
    print(f"Total missing words: {len(missing_words)}")
    print("Sample missing words:", missing_words[:10])
    
    # Verwijder woorden die niet in het embeddingmodel zitten
    filtered_lemmas = [word for word in unique_lemmas if word in embedding_model]

    def filter_adjectives(lemmas, nlp):
        """
        Filtert de lijst met lemmas op Nederlandse adjectieven met behulp van spaCy.
        """
        adj_set = set()
        for lemma in lemmas:
            doc = nlp(lemma)
            if doc and doc[0].pos_ == 'ADJ' and lemma in embedding_model:
                adj_set.add(lemma)
        return list(adj_set)
    
    # Filter op adjectieven
    filtered_adjectives = filter_adjectives(filtered_lemmas, nlp_model)
    
    # Filter lemmas containing target words explicitly
    target_words = [
        'man','mannen','jongen','kerel','vader','zoon',
        'vrouw','vrouwelijk','vrouwen','meisje','dame'
    ]

    # Exclude adjectives containing target words as substrings
    filtered_adjectives = [
        adj for adj in filtered_adjectives
        if not any(target_word in adj for target_word in target_words)
    ]

    print(f"Remaining adjectives after final filtering: {len(filtered_adjectives)}")
    
    return filtered_adjectives
    

In [24]:
adjectives = get_clean_adjectives(
    '/Users/matthijstentije/University/MSc_Data-Science/Thesis/MSc_Data_Science_Thesis/data/ANW_resultaten(a-o).csv',
    '/Users/matthijstentije/University/MSc_Data-Science/Thesis/MSc_Data_Science_Thesis/data/ANW_resultaten(o-z).csv',
    embedding_model=nl_embeddings,
    nlp_model=nlp
)
print("Gefilterde adjectieven:", adjectives)

Total unique lemmas: 7500
Total missing words: 1533
Sample missing words: ['aalglad', 'aalsmeerder', 'aalsmeers', 'aalsts', 'aaltens', 'aalvlug', 'aandachtsgeil', 'aandeelhoudend', 'aangedampt', 'aanhalerig']
Remaining adjectives after final filtering: 3645
Gefilterde adjectieven: ['vormeloos', 'gezellig', 'kaukasisch', 'hoornachtig', 'onaanvaardbaar', 'oprolbaar', 'diep', 'varenachtig', 'zandkleurig', 'syndicaal', 'gedragswetenschappelijk', 'luchtledig', 'veelkoppig', 'voltijds', 'bitter', 'brandveilig', 'luizig', 'minderjarig', 'spookachtig', 'waardeerbaar', 'echt', 'orthodox', 'vraatzuchtig', 'neerwaarts', 'geldelijk', 'expliciet', 'onrechtstreeks', 'verkieslijk', 'absurd', 'milieuvervuilend', 'groenig', 'moederlijk', 'gerust', 'hoopvol', 'strak', 'korrelig', 'duinachtig', 'kosmologisch', 'onbeleefd', 'honderdvoudig', 'netelig', 'prekerig', 'lustig', 'benauwd', 'wild', 'kafka', 'wisselend', 'stressvol', 'goedaardig', 'lijdelijk', 'zuurvrij', 'tam', 'dunbevolkt', 'effectief', 'eerloo

In [25]:
# Representatieve concepten als spaCy tokens
persoon_concepten = [nlp(w)[0] for w in ["persoon", "mens", "man", "vrouw", "kind"]]

def kan_persoon_beschrijven(adj, min_gelijkenis=0.3):
    adj_token = nlp(adj)[0]

    if not adj_token.has_vector:
        return False

    # Gemiddelde gelijkenis met persoonsconcepten
    gelijkenissen = [
        adj_token.similarity(concept_token)
        for concept_token in persoon_concepten
    ]

    gemiddelde_gelijkenis = sum(gelijkenissen) / len(gelijkenissen)

    return gemiddelde_gelijkenis >= 0.3  # Drempel aanpassen naar behoefte


resultaten = {adj: kan_persoon_beschrijven(adj) for adj in adjectives}

print(resultaten)
print("Aantal adjectieven:", len(resultaten))
print("Aantal geschikt voor persoon:", sum(resultaten.values()))

# Maak nieuwe lijst met alleen geschikte adjectieven
adjectives = [adj for adj, is_geschikt in resultaten.items() if is_geschikt]

print(adjectives)


{'vormeloos': False, 'gezellig': False, 'kaukasisch': False, 'hoornachtig': False, 'onaanvaardbaar': False, 'oprolbaar': False, 'diep': False, 'varenachtig': False, 'zandkleurig': False, 'syndicaal': False, 'gedragswetenschappelijk': False, 'luchtledig': False, 'veelkoppig': False, 'voltijds': False, 'bitter': False, 'brandveilig': False, 'luizig': False, 'minderjarig': True, 'spookachtig': False, 'waardeerbaar': False, 'echt': False, 'orthodox': False, 'vraatzuchtig': True, 'neerwaarts': False, 'geldelijk': True, 'expliciet': False, 'onrechtstreeks': False, 'verkieslijk': True, 'absurd': False, 'milieuvervuilend': False, 'groenig': False, 'moederlijk': True, 'gerust': False, 'hoopvol': False, 'strak': False, 'korrelig': False, 'duinachtig': False, 'kosmologisch': False, 'onbeleefd': True, 'honderdvoudig': False, 'netelig': False, 'prekerig': False, 'lustig': False, 'benauwd': False, 'wild': False, 'kafka': False, 'wisselend': False, 'stressvol': False, 'goedaardig': True, 'lijdelijk':

In [26]:
def compute_individual_bias(adjectives, male_terms, female_terms, model):
    """
    Voor elk bijvoeglijk naamwoord wordt het verschil berekend tussen de gemiddelde cosine similarity
    met mannelijke termen en met vrouwelijke termen. Woorden waarin een van de target-woorden voorkomt 
    (bijvoorbeeld 'dame' in 'damesachtig') worden eerst uitgesloten.
    """
    # Combineer de target-woorden en filter de adjectives
    target_words = male_terms + female_terms
    filtered_adjectives = [adj for adj in adjectives if not any(target in adj for target in target_words)]
    
    bias_results = {}
    for adj in filtered_adjectives:
        # Controleer of de embedding voor het bijvoeglijk naamwoord beschikbaar is
        if model[adj] is None:
            continue
        
        male_sims = []
        for term in male_terms:
            vec_term = model[term]
            if vec_term is None:
                continue
            male_sims.append(cosine_similarity(model[adj], vec_term))
        
        female_sims = []
        for term in female_terms:
            vec_term = model[term]
            if vec_term is None:
                continue
            female_sims.append(cosine_similarity(model[adj], vec_term))
        
        # Bereken de bias alleen als er zowel mannelijke als vrouwelijke vergelijkingen beschikbaar zijn
        if male_sims and female_sims:
            bias_value = np.mean(male_sims) - np.mean(female_sims)
            bias_results[adj] = bias_value
    return bias_results

# Voorbeeldlijsten target woorden
target_words_man = ['man','mannen','jongen','kerel','vader','zoon']
target_words_vrouw = ['vrouw','vrouwelijk','vrouwen','meisje','dame']

# Aannemende dat `unique_lemmas` je lijst met schoongemaakte lemma's i 
# en `model` jouw word embedding model (bijv. een WEFE-wrapped FastText-model)
individual_bias = compute_individual_bias(adjectives=adjectives, male_terms=target_words_man, female_terms=target_words_vrouw, model=model)

# Sorteer de bias-resultaten op bias-waarde (van laag naar hoog)
sorted_bias = sorted(individual_bias.items(), key=lambda x: x[1])

# Selecteer de 30 laagste scores
lowest_30 = sorted_bias[:30]

# Selecteer de 30 hoogste scores
highest_30 = sorted_bias[-30:]

print("30 Laagste Scores:")
for word, score in lowest_30:
    print(f"{word}: {score:.3f}")

print("\n30 Hoogste Scores:")
for word, score in reversed(highest_30):
    print(f"{word}: {score:.3f}")



30 Laagste Scores:
seksistisch: -0.123
cisgender: -0.123
beeldschoon: -0.120
begeerlijk: -0.116
verloskundig: -0.106
verleidelijk: -0.106
gedistingeerd: -0.104
paradijselijk: -0.104
maagdelijk: -0.104
verlokkelijk: -0.103
zinnelijk: -0.098
seksueel: -0.095
liefelijk: -0.094
geslachtelijk: -0.094
seksloos: -0.092
halfnaakt: -0.092
intersekse: -0.088
lieflijk: -0.087
levensbeschouwelijk: -0.086
mysterieus: -0.085
haarloos: -0.083
intermenselijk: -0.080
liefdadig: -0.080
ongeslachtelijk: -0.077
onzedelijk: -0.076
onnatuurlijk: -0.074
indisch: -0.073
onpersoonlijk: -0.072
schrijnend: -0.072
tandeloos: -0.072

30 Hoogste Scores:
dood: 0.101
adoptief: 0.074
gebroederlijk: 0.072
slaapdronken: 0.071
kameraadschappelijk: 0.067
schielijk: 0.064
broederlijk: 0.059
overmoedig: 0.058
ouderlijk: 0.055
godsgruwelijk: 0.053
gehoorzaam: 0.053
crimineel: 0.052
plotseling: 0.052
meerderjarig: 0.051
moederloos: 0.051
onbedaarlijk: 0.051
dolgelukkig: 0.051
ontroostbaar: 0.050
onnoemelijk: 0.050
krankzinnig

In [27]:
# Lijst met specifieke bijvoeglijke naamwoorden waarin je geïnteresseerd bent
search_words = ["sterk", "zacht", "moedig", "emotioneel", "dominant", "zorgzaam", "aardig", "knap", "schattig"]

print("Bias scores voor specifieke woorden:")
for word in search_words:
    # Haal de bias score op, indien het woord aanwezig is in de bias resultaten
    bias_value = individual_bias.get(word)
    if bias_value is not None:
        print(f"{word}: {bias_value:.3f}")
    else:
        print(f"{word}: Niet gevonden in de bias resultaten.")


Bias scores voor specifieke woorden:
sterk: Niet gevonden in de bias resultaten.
zacht: Niet gevonden in de bias resultaten.
moedig: 0.030
emotioneel: -0.032
dominant: Niet gevonden in de bias resultaten.
zorgzaam: -0.059
aardig: Niet gevonden in de bias resultaten.
knap: Niet gevonden in de bias resultaten.
schattig: Niet gevonden in de bias resultaten.


In [28]:
from wefe.metrics import RIPA
from wefe.query import Query

# Debugging print statements
print(f"Number of adjectives (filtered_lemmas): {len(adjectives)}")
print(f"Number of Male Terms: {len(['man', 'mannelijk', 'hij', 'hem', 'kerel', 'jongen', 'mannen'])}")
print(f"Number of Female Terms: {len(['vrouw', 'vrouwelijk', 'zij', 'haar', 'dame', 'meisje', 'vrouwen'])}")

# Define the query
query = Query(
    target_sets=[
        ['man', 'mannelijk', 'hij', 'hem', 'kerel', 'jongen', 'mannen'],
        ['vrouw', 'vrouwelijk', 'zij', 'haar', 'dame', 'meisje', 'vrouwen']
    ],
    attribute_sets=[adjectives],  # Ensure it's a list of lists
    target_sets_names=["Male Terms", "Female Terms"],
    attribute_sets_names=["Adjectives"],
)

# Instantiate the metric and run the query
ripa = RIPA()
result = ripa.run_query(query, model)

# Print results
print(result)

Number of adjectives (filtered_lemmas): 587
Number of Male Terms: 7
Number of Female Terms: 7
{'query_name': 'Male Terms and Female Terms wrt Adjectives', 'result': 0.025646292, 'ripa': 0.025646292, 'word_values': {'minderjarig': {'mean': 0.035048593, 'std': 0.055665776}, 'vraatzuchtig': {'mean': 0.018955585, 'std': 0.036924157}, 'geldelijk': {'mean': 0.008722569, 'std': 0.052882794}, 'verkieslijk': {'mean': 0.032397904, 'std': 0.042957325}, 'moederlijk': {'mean': 0.0012976801, 'std': 0.05408321}, 'onbeleefd': {'mean': 0.038742628, 'std': 0.055873506}, 'goedaardig': {'mean': 0.045166165, 'std': 0.05672968}, 'lijdelijk': {'mean': 0.028692853, 'std': 0.04517965}, 'eerlijk': {'mean': 0.05756527, 'std': 0.07210884}, 'schichtig': {'mean': 0.026092773, 'std': 0.06613627}, 'seksistisch': {'mean': 0.00052079506, 'std': 0.047304355}, 'hersenloos': {'mean': 0.059173733, 'std': 0.044781066}, 'zelfzuchtig': {'mean': 0.02633163, 'std': 0.03297346}, 'geriefelijk': {'mean': 0.008378563, 'std': 0.0391

In [30]:
import pandas as pd

# Extract scores into a DataFrame
scores_df = pd.DataFrame({
    'Word': result['word_values'].keys(),
"Mean Score": [value['mean'] for value in result['word_values'].values()],
    "Std Dev": [value['std'] for value in result['word_values'].values()]
})

# Sort for readability
scores_df = scores_df.sort_values(by='Mean Score', ascending=False).reset_index(drop=True)
scores_df.head(n=50)

,Word,Mean Score,Std Dev
0,dood,0.145030,0.116842
1,nuchter,0.088686,0.070261
2,ziek,0.088313,0.126671
3,moedig,0.080407,0.079138
4,gelijk,0.079987,0.068564
5,schielijk,0.071583,0.066088
6,goedmoedig,0.069009,0.059942
7,overmoedig,0.067355,0.037127
8,crimineel,0.065246,0.060519
9,strijdlustig,0.064356,0.041580
